##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [1]:
import tensorflow as tf

If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist # a data set known to Keras/TensorFlow
# mnist.load_data() produces a pair of input/output tensors: one for training
# and one for testing.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # scale down input
x_train, y_train = x_train[:6000], y_train[:6000]
print("===========x_train===========")
print(tf.shape(x_train))
print("===========x_train element 0 (28 rows of 28 gray values)===========")
print(x_train[0])  # for inspection
print("===========y_train===========")
print(tf.shape(y_train))  # format of output data: 60K records 0-9
print(y_train)  # for inspection

===========x_train===========
tf.Tensor([6000   28   28], shape=(3,), dtype=int32)
===========x_train element 0 (28 rows of 28 gray values)===========
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [3]:
model = tf.keras.models.Sequential([  # layer format for the neural net
  # each pixel (grayscale value) mapped to one of 784 nodes
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  # fully connected to hidden layer with relu
  tf.keras.layers.Dense(1024, activation='relu'),
  # Dropout layer randomly sets its input units to 0 at 20% rate at each training step
  # The other inputs are scaled up by 1/0.8 so sum over all inputs is unchanged
  # Illustrative figure: http://laid.delanover.com/wp-content/uploads/2018/02/dropout.png
  # (Doc: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)
  tf.keras.layers.Dropout(0.20),
  tf.keras.layers.Dense(10) # e.g., output #7 = degree to which the output is 7
])

c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.04044887, -0.35165915, -0.06214158, -0.14889549, -0.5873819 ,
         0.21824503, -0.07997231, -0.51219726,  0.4353646 ,  0.02084406]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.10266964, 0.07521165, 0.10046644, 0.09211794, 0.05941718,
        0.13298152, 0.09869092, 0.06405666, 0.16522859, 0.10915943]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.017545

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=15)

Epoch 1/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7752 - loss: 0.7742
Epoch 2/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9467 - loss: 0.2134
Epoch 3/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9651 - loss: 0.1256
Epoch 4/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9786 - loss: 0.0785
Epoch 5/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9858 - loss: 0.0619
Epoch 6/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9899 - loss: 0.0382
Epoch 7/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9939 - loss: 0.0260
Epoch 8/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9964 - loss: 0.0195
Epoch 9/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9971 - loss: 0.0155
Epoch 10/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9976 - loss: 0.0130
Epoch 11/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9978 - loss: 0.0107
Epoch 12/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step

The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 1ms/step - accuracy: 0.9532 - loss: 0.1884


[0.1884218454360962, 0.9531999826431274]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.5427084e-07, 7.5027339e-11, 3.1343968e-06, 4.6326011e-05,
        8.9990645e-13, 1.5266414e-07, 2.0456495e-13, 9.9994934e-01,
        2.1858141e-07, 7.8530559e-07],
       [1.7771253e-08, 8.4947787e-05, 9.9989736e-01, 1.8331458e-06,
        8.3026415e-15, 3.4793761e-08, 6.2480517e-08, 7.9319728e-12,
        1.5733674e-05, 3.5440111e-15],
       [3.3602277e-07, 9.9804592e-01, 5.7737787e-05, 5.8099768e-06,
        6.3599517e-05, 2.3768812e-06, 2.3459031e-06, 1.6781164e-03,
        1.4260423e-04, 1.0617223e-06],
       [9.9997318e-01, 8.7561805e-11, 9.8983969e-07, 9.0878149e-10,
        1.0803159e-07, 3.3033965e-07, 2.3622524e-05, 6.7567669e-07,
        4.7765565e-09, 1.1131582e-06],
       [1.1404303e-05, 2.4895797e-09, 6.0792736e-06, 4.9114313e-08,
        9.9449867e-01, 5.9158310e-06, 2.9842497e-05, 4.8574508e-05,
        8.5193824e-06, 5.3910208e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
